## Pre-processing Images (color)

In [ ]:
"""
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Read the images
img = cv2.imread("IAM Dataset/forms/a01-000u.png")

# Convert Image to Image HSV
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Defining lower and upper bound HSV values
lower = np.array([0, 0, 30])
upper = np.array([255, 100, 100])


# Defining mask for detecting color
mask = cv2.inRange(hsv, lower, upper)

plt.show()
"""

## Pre-processing Images (lines)

In [ ]:
import cv2
import numpy as np


# Reading the required image in
# which operations are to be done.
# Make sure that the image is in the same
# directory in which this python program is
img = cv2.imread("IAM Dataset/forms/a01-000u.png")

# Convert image to grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Use canny edge detection
edges = cv2.Canny(gray,50,150,apertureSize=3)

# Apply HoughLinesP method to
# to directly obtain line end points
lines_list =[]
lines = cv2.HoughLinesP(
            edges, # Input edge image
            1, # Distance resolution in pixels
            np.pi/180, # Angle resolution in radians
            threshold=100, # Min number of votes for valid line
            minLineLength=5, # Min allowed length of line
            maxLineGap=10 # Max allowed gap between line for joining them
            )

# Iterate over points
for points in lines:
      # Extracted points nested in the list
    x1,y1,x2,y2=points[0]
    # Draw the lines joing the points
    # On the original image
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
    # Maintain a simples lookup list for points
    lines_list.append([(x1,y1),(x2,y2)])


In [ ]:
import matplotlib.pyplot as plt

img = cv2.imread("IAM Dataset/forms/a01-000u.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

gray = cv2.bitwise_not(gray)
bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, \
                                cv2.THRESH_BINARY, 15, -2)

horizontal=np.copy(bw)

horizontalStructure=cv2.getStructuringElement(cv2.MORPH_RECT, (200,1))
horizontal = cv2.erode(horizontal, horizontalStructure)
horizontal = cv2.dilate(horizontal, horizontalStructure)

plt.imshow(horizontal)
plt.show()

## Page Segmentation

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
def findHorizontalLines(img):
    img = cv2.imread(img)

    #convert image to greyscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # set threshold to remove background noise
    thresh = cv2.threshold(gray,30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


    # define rectangle structure (line) to look for: width 100, hight 1. This is a
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (200,1))

    # Find horizontal lines
    lineLocations = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)

    return lineLocations


ll=findHorizontalLines("IAM Dataset/forms/a01-000u.png")
plt.imshow(ll)
plt.show()

In [ ]:
import pandasql as ps
import pandas as pd
def linePosition(lineLoc):
    df_lineLocations = pd.DataFrame(lineLoc.sum(axis=1)).reset_index()
    df_lineLocations.columns = ['rowLoc', 'LineLength']
    df_lineLocations['line'] = 0
    df_lineLocations['line'][df_lineLocations['LineLength'] > 100] = 1
    df_lineLocations['cumSum'] = df_lineLocations['line'].cumsum()



    query = '''
    select row_number() over (order by cumSum) as SegmentOrder
    , min(rowLoc) as SegmentStart
    , max(rowLoc) - min(rowLoc) as Height
    from df_lineLocations
    where line = 0
    --and CumSum !=0
    group by cumSum
    '''

    df_SegmentLocations  = ps.sqldf(query, locals())
    return df_SegmentLocations

lp=linePosition(ll)
lp

## Generate Dataset without noise

In [ ]:
import os
def pageSegmentation1(img):
    lineLocations = findHorizontalLines(img)
    w = lineLocations.shape[1]
    df_SegmentLocations = linePosition(lineLocations)
    img = cv2.imread(img)
    im2 = img.copy()
    i = df_SegmentLocations['Height'].idxmax()
    y = df_SegmentLocations['SegmentStart'][i]
    h = df_SegmentLocations['Height'][i]
    cropped = im2[y:y + h, 0:w]

    return cropped



directory = "IAM Dataset/forms"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        segment = pageSegmentation1(f)
        cv2.imwrite("IAM Dataset/formsSegment/"+filename, segment)

In [ ]:
l_img = np.zeros((2500,2500,3), np.uint8)
l_img.fill(255)
plt.show

## Squaring Dataset

In [ ]:

def rectangImage(img):
    l_img = np.zeros((2500,2500,3), np.uint8)
    l_img.fill(255)
    s_img = cv2.imread(img)
    l_img[0:s_img.shape[0], 0:s_img.shape[1]] = s_img
    return l_img
im=rectangImage("IAM Dataset/formsSegment/a01-000u.png")
plt.imshow(im)


In [ ]:
directory = "IAM Dataset/formsSegment"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        image = rectangImage(f)
        cv2.imwrite("IAM Dataset/rectFormsSegment/"+filename, image)

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

## Reading file forms, containing the info of the images in IAM Dataset

In [ ]:
data = pd.read_csv('IAM Dataset/forms.txt', sep=" ", header=None)
data.columns = ["form id","writerId","nSentences","wordSegmentation","nLine","nLineCorrectlySegmented","nWord","nWordCorrectlySegmented"]
data

## Drop all unused features

In [ ]:
data=data.drop(["nSentences","wordSegmentation","nLine","nLineCorrectlySegmented","nWord","nWordCorrectlySegmented"], axis=1)
data.nunique()

## How many writers we have and how many text wrote each one of them

In [ ]:
data["writerId"].value_counts()

In [ ]:
data["writerId"].value_counts().value_counts()

In [ ]:
#set the number to select only writers with more than X written
minimumNumPaperForEachWriter=4
is_multi = data["writerId"].value_counts() >= minimumNumPaperForEachWriter
filtered = data[data["writerId"].isin(is_multi[is_multi].index)]
NWriters=len(filtered["writerId"].value_counts())

In [ ]:
#Create list of writers with less than X papers
is_multi = data["writerId"].value_counts() < minimumNumPaperForEachWriter
filtered = data[data["writerId"].isin(is_multi[is_multi].index)]
lw =filtered["writerId"].tolist()
print(len(lw))
lw = list(dict.fromkeys(lw))
len(lw)

In [ ]:
dataWriterNotRemoved = pd.DataFrame(data[data["writerId"].isin(lw)==False])
dataWriterNotRemoved

In [ ]:
NWriters=len(dataWriterNotRemoved["writerId"].value_counts())
NWriters

## Resizing and Thresholding function

In [ ]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [ ]:
def thresholding(img):
    img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(img_gray,80,255,cv2.THRESH_BINARY_INV)
    return thresh

## Set Img Size than load the data of selected writer

In [ ]:
SizeImg=500
def loadData():
    listWriters = dataWriterNotRemoved["writerId"].tolist()
    imgArray = []

    for i, row in dataWriterNotRemoved.iterrows():
        im=cv2.imread("IAM Dataset/rectFormsSegment/"+row["form id"]+".png")
        im=image_resize(im,SizeImg)
        im=thresholding(im)
        imgArray.append(im)
    return imgArray, listWriters

In [ ]:
X, y = loadData()
print(len(X))
print(len(y))

## CNN

In [ ]:
import tensorflow
import PIL.ImageOps

from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential, Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Add
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint



def halfDeepWriter(input_shape, classes, frac=1):
    patch_1 = Input(shape=input_shape)

    out1 = Conv2D(int(96*frac), kernel_size=5, strides=2, activation='relu')(patch_1)
    out1 = MaxPooling2D(3, strides=2)(out1)

    out1 = Conv2D(int(256*frac), kernel_size=3, activation='relu')(out1)
    out1 = MaxPooling2D(3, strides=2)(out1)

    out1 = Conv2D(int(384*frac), kernel_size=3, activation='relu')(out1)
    out1 = Conv2D(int(384*frac), kernel_size=3, activation='relu')(out1)
    out1 = Conv2D(int(256*frac), kernel_size=3, activation='relu')(out1)
    out1 = MaxPooling2D(3, strides=2)(out1)

    out1 = Flatten()(out1)
    out1 = Dense(int(1024*frac), activation='relu')(out1)
    out1 = Dropout(0.5)(out1)

    out1 = Dense(int(1024*frac), activation='relu')(out1)
    out1 = Dropout(0.5)(out1)

    out1 = Dense(classes, activation='softmax')(out1)

    model = Model(inputs=patch_1, outputs=out1)
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['acc'])

    return model

In [ ]:
class dataGeneratorHalfDeepWriter(tensorflow.keras.utils.Sequence):
    def __init__(self, X, y, batch_size=32, shuffle=True, w=80):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.inputX = X
        self.inputY = y
        self.w = w
        self.h = self.inputX[0].shape[0]
        self.total = len(X)
        self.indexes = np.arange(self.total)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.total / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        return self.__data_generation(indexes)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batchIndexes):
        'Generates data containing batch_size samples' # X : (2, n_samples, *dim, n_channels)
        # Initialization
        X = np.zeros((self.batch_size, self.h, self.w))
        y = np.empty((self.batch_size, self.inputY.shape[-1]), dtype=int)

        # Generate data
        for i, ID in enumerate(batchIndexes):
            # Black Image
            tmpImg = np.zeros((self.h, self.w))

            # Starting column position
            y_pos1 = int(np.random.randint(low=0,
                        high=max(self.inputX[ID].shape[1]-self.w//3, 1),
                        size=1))

            # Placing Image in black image
            tmpImg1 = (self.inputX[ID])[:, y_pos1:y_pos1+self.w]

            # Placing Image in output
            X[i, 0:tmpImg1.shape[0], 0:tmpImg1.shape[1]] = tmpImg1

            # Store class
            y[i] = self.inputY[ID]

        X = X[:, :, :, np.newaxis]
        return X, y

In [ ]:
model = halfDeepWriter((SizeImg, SizeImg, 1), NWriters)
model.summary()

## Training

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from sklearn.preprocessing import OneHotEncoder


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,
                                                    random_state=40)

OHE = OneHotEncoder().fit(np.array(y).reshape(-1, 1))

y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_train_OHE = OHE.fit_transform(y_train).toarray()
y_test_OHE = OHE.transform(y_test).toarray()

train_gen = dataGeneratorHalfDeepWriter(X_train, y_train_OHE, batch_size=32, w=SizeImg)
test_gen = dataGeneratorHalfDeepWriter(X_test, y_test_OHE, batch_size=32, w=SizeImg)



In [ ]:
hist = model.fit(train_gen, validation_data=test_gen, epochs=200, callbacks=[ ModelCheckpoint(filepath='content/best.hdf5',
                             save_best_only=True, monitor='acc', mode='max',
                            ), ])

## Resume from checkpoint

In [ ]:
model_weights_path = "./best.hdf5"
if model_weights_path:
    model.load_weights(model_weights_path)
    scores = model.evaluate_generator(test_gen, steps=round(len(X_test)/32))
    print("Accuracy: ", scores[1])
else:
    print("Set model weights file to load in the 'model_weights_path' variable")